In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.stem import WordNetLemmatizer 
import string
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
df = pd.read_csv('../data/mostly_cleaned.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48890 entries, 0 to 48889
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            48890 non-null  int64  
 1   age                   48890 non-null  int64  
 2   status                48890 non-null  object 
 3   sex                   48890 non-null  object 
 4   orientation           48890 non-null  object 
 5   body_type             48890 non-null  object 
 6   diet                  48890 non-null  object 
 7   drinks                48890 non-null  object 
 8   drugs                 48890 non-null  object 
 9   education             48890 non-null  object 
 10  ethnicity             44945 non-null  object 
 11  height                48890 non-null  float64
 12  job                   48890 non-null  object 
 13  offspring             48890 non-null  object 
 14  pets                  48890 non-null  object 
 15  religion           

In [4]:
essays = df.iloc[:, 19:29]
essays.fillna(".", inplace=True)
essays['essays_combined'] = essays['essay0']
for col in essays.columns[1:]:
    essays['essays_combined'] += essays[col]

In [5]:
trimmed_df = df.drop(columns={'Unnamed: 0', 
                 'sign', 
                 'religion', 
                 'essay0', 
                 'essay1', 
                 'essay2', 
                 'essay3', 
                 'essay4', 
                 'essay5', 
                 'essay6', 
                 'essay7', 
                 'essay8', 
                 'essay9',
                 'sign_actual'})

In [6]:
dummy_df = pd.get_dummies(trimmed_df)

In [7]:
list(dummy_df.columns)

['age',
 'height',
 'status_available',
 'status_married',
 'status_seeing someone',
 'status_single',
 'status_unknown',
 'sex_f',
 'sex_m',
 'orientation_bisexual',
 'orientation_gay',
 'orientation_straight',
 'body_type_a little extra',
 'body_type_athletic',
 'body_type_average',
 'body_type_curvy',
 'body_type_fit',
 'body_type_full figured',
 'body_type_jacked',
 'body_type_no_answer',
 'body_type_overweight',
 'body_type_rather not say',
 'body_type_skinny',
 'body_type_thin',
 'body_type_used up',
 'diet_anything',
 'diet_halal',
 'diet_kosher',
 'diet_mostly anything',
 'diet_mostly halal',
 'diet_mostly kosher',
 'diet_mostly other',
 'diet_mostly vegan',
 'diet_mostly vegetarian',
 'diet_no_answer',
 'diet_other',
 'diet_strictly anything',
 'diet_strictly halal',
 'diet_strictly kosher',
 'diet_strictly other',
 'diet_strictly vegan',
 'diet_strictly vegetarian',
 'diet_vegan',
 'diet_vegetarian',
 'drinks_desperately',
 'drinks_no_answer',
 'drinks_not at all',
 'drinks_o

In [8]:
for col in list(dummy_df.columns):
    if dummy_df[col].dtype != 'uint8':
        print(col, dummy_df[col].dtype)

age int64
height float64


In [9]:
tfidf = TfidfVectorizer(max_features=1000, 
                        stop_words='english', 
                        strip_accents='unicode', 
                        analyzer='word')
Z = tfidf.fit_transform(essays.essays_combined)

In [10]:
vector_df = pd.DataFrame(Z.toarray())

In [11]:
vector_df.iloc[0:10,0:1000].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 1000 entries, 0 to 999
dtypes: float64(1000)
memory usage: 78.2 KB


In [12]:
vector_df[0:1]

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.173365,0.0,0.0,...,0.0,0.0,0.0,0.064153,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
combined_df = dummy_df.join(vector_df)

In [14]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48890 entries, 0 to 48889
Columns: 8059 entries, age to 999
dtypes: float64(1001), int64(1), uint8(7057)
memory usage: 702.8 MB


In [15]:
y=df['sign_actual']
X=combined_df

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36667, 8059), (12223, 8059), (36667,), (12223,))

In [17]:
LR = LogisticRegression(multi_class='multinomial', max_iter=1000)
LR.fit(X_train, y_train)
LR_pred = LR.predict(X_test)

LR.score(X_test, y_test)

KeyboardInterrupt: 

In [19]:
GB = GradientBoostingClassifier()
GB.fit(X_train, y_train)
GB_pred = GB.predict(X_test)

GB.score(X_test, y_test)

0.0926122883089258

In [20]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)
RF_pred = RF.predict(X_test)

RF.score(X_test, y_test)

0.08279473124437536

In [ ]:
gb_gridsearch = GridSearchCV(estimator=GradientBoostingClassifier(),
                                param_grid = {'max_depth': [2, 4, 8],
                                             'min_samples_split': [2, 4, 8],
                                             'subsample': [1, 0.5, 0.25, 0.1]},
                                cv=5, verbose=0, return_train_score=True)
gb_gridsearch.fit(X_train, y_train)
df_gridsearch = pd.DataFrame(gb_gridsearch.cv_results_)